In [ ]:
%matplotlib inline
#%pylab
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as mpc
import matplotlib.dates as dts
import numpy as np
import pandas as pd
import itertools
import os
import ROOT
import datetime
from root_numpy import root2array, root2rec, tree2rec, array2root
from scipy.optimize import curve_fit
from scipy.misc import factorial

In [ ]:
FIGPATH = "./figs/tagged/"
# FIGPATH = "./figs/no-cali/tagged/"
PLOTSTATUS = "[Fake Data]"
# PLOTSTATUS = "[Run4a EXT No eLifetime.]"

# truncated last bin to use for Landau-Gaussian fit range tweaking.
FIT_TRUNC = None
# FIT_TRUNC = 20
print  min(np.linspace(1,6.0,50)[0:FIT_TRUNC]), " " ,max(np.linspace(1, 6.0, 50)[0:FIT_TRUNC])

In [ ]:
plt.rcParams.update({"font.size": 18})
# matplotlib.rcParams['backend'] = 'cairo'

In [ ]:
from ELOSS import dpdx as DPDX
from ELOSS import dedx as DEDX

In [ ]:
from scipy.interpolate import spline

pdg_rr_v = []
pdg_dedx_v = []
pdg_ke_v = []
pdg_rr_fit_v = []
pdg_dedx_fit_v = []
pdg_ke_fit_v = []

# fin = open('/home/david/Neutrinos/StopMuCalibration/mutable.txt')
fin = open('./data/mutable.txt')
rho = 1.396

for line in fin:
    words = line.split()
    pdg_rr_v.append(   float(words[-1])/rho )
    pdg_dedx_v.append( float(words[2]) *rho )
    pdg_ke_v.append(   float(words[0])      )


pdg_rr_fit_v   = np.linspace(1,1000,1000)
pdg_dedx_fit_v = spline(pdg_rr_v,pdg_dedx_v,pdg_rr_fit_v)
pdg_ke_fit_v = spline(pdg_rr_v,pdg_ke_v,pdg_rr_fit_v)


# fiven a RR value, get the energy
def ERange(RR):
    for i in xrange(len(pdg_rr_fit_v)):
        if (RR <= pdg_rr_fit_v[i]):
            return pdg_ke_fit_v[i]              
    return -1

print ERange(100.)

In [ ]:
#df = pd.read_pickle('stopmu.pkl')
#df = pd.read_pickle('stopmu_data_mcc83.pkl') # data
data = pd.read_csv('stopmu_mcc9_data.csv',sep='\t')
print data.shape

In [ ]:
data.columns = ["rr", "dqdx", "pitch", "px", "py", "pz"]

In [ ]:
fig = plt.figure(figsize=(12, 6))
BINS = (np.linspace(0, 250, 100), np.linspace(150, 500, 100))
plt.hist2d(data["rr"].values, data["dqdx"].values, bins=BINS)
plt.grid()
plt.xlabel("Residual Range [cm]", fontsize=20, fontweight="bold")
plt.ylabel("dQ/dx [ADC / cm]", fontsize=20, fontweight="bold")
# plt.title('Tagged Stopping Muon Profile [MC Cosmics]',fontsize=20,fontweight='bold')
plt.title(
    "Tagged Stopping Muon Profile [DATA]", fontsize=18, fontweight="bold", loc="left"
)
plt.title(PLOTSTATUS, fontsize=14, loc="right")
plt.savefig(FIGPATH + "stopping_mu_rr_profile.pdf", bbox_inches="tight")
plt.savefig(FIGPATH + "stopping_mu_rr_profile.png", bbox_inches="tight")
plt.show()

In [ ]:
# invert Recombination Modified Box Model to get dE/dx from dQ/dx

# argon density [g/cm^3]
rho = 1.396
# electric field [kV/cm]
efield = 0.273
# ionization energy [MeV/e]
Wion = 23.6 * (10 ** (-6))

fModBoxA = 0.93
fModBoxB = 0.562


def ModBoxInverse(dqdx):
    dedx = (np.exp(fModBoxB * Wion * dqdx) - fModBoxA) / fModBoxB
    return dedx

In [ ]:
def dEdx(x, elecgain):
    dqdx = x["dqdx"] * elecgain
    return ModBoxInverse(dqdx)

In [ ]:
# fin = open('/home/david/Neutrinos/StopMuCalibration/mutable.txt')
fin = open("./data/mutable.txt")
rho = 1.396
pdg_rr_v = []
pdg_dedx_v = []
for line in fin:
    words = line.split()
    pdg_rr_v.append(float(words[-1]) / rho)
    pdg_dedx_v.append(float(words[2]) * rho)

In [ ]:
from scipy.interpolate import spline

pdg_rr_fit_v = np.linspace(1, 1000, 1000)
pdg_dedx_fit_v = spline(pdg_rr_v, pdg_dedx_v, pdg_rr_fit_v)

In [ ]:
# pylandau from https://github.com/SiLab-Bonn/pylandau
from pylandau import langau
from scipy.optimize import curve_fit

guess = [4.0, 0.2, 0.2]  # MPV, Landau sigma, gauss sigma in that order


def GL(x_v, mpv, sL, sG, A):
    return A * langau(x_v * 100.0, mpv * 100, sL * 100, sG * 100)

Compute and minimise $\chi^2$ as a function of electronics gain. Described as:
$$
        \chi^2 = \frac{1}{N} \frac{\mbox{(MPV theory - MPV measured (gain))}^2}{\sigma^2}
$$

In [ ]:
# calculate chi^2 of PDG - data as a function of electronics gain applied

elec_gain = np.linspace(230,250,20) # nominal run 4a range
# elec_gain = np.linspace(240,260,20) # no e-lifetime range
# elec_gain = np.linspace(250,270,20)
chisq_v  = []

rr_ranges = np.linspace(100,150,10)

BINS = np.linspace(1,6.0,100)
# BINS = np.linspace(1,6.0,50)

datafit = data.query('rr > 100 and rr < 150 and pitch > 0.3 and pitch < 0.4 and px < 0.2 and px >-0.2')
print datafit.shape

for elecgain in elec_gain:
    
    datafit['dedx'] = datafit.apply(lambda x : dEdx(x,elecgain),axis=1)
    
    chilocal = 0.
    
    for n in xrange(len(rr_ranges)-1):
        
        rrmin = rr_ranges[n]
        rrmax = rr_ranges[n+1]
        
        if (n == 1) or (n == 3):
#             print "skipping"
            continue
        
        rravg = 0.5*(rrmax+rrmin)
        
        dftmp = datafit.query('rr > %i and rr < %i'%(rrmin,rrmax))
        
        dedx_v = dftmp['dedx'].values
    
        vals,bine = np.histogram(dedx_v,bins=BINS)
        binc = 0.5*(bine[1:]+bine[:-1])
        guess = [1.8,0.1,0.1,30.]
#         popt,popv = curve_fit(GL,binc,vals,p0=guess,bounds=([1.2,0.02,0.02,1],[4.0,0.3,0.3,200]))#,sigma=np.sqrt(vals),absolute_sigma=True)
        popt,popv = curve_fit(GL,binc[0:FIT_TRUNC],vals[0:FIT_TRUNC],p0=guess,bounds=([1.2,0.02,0.02,1],[4.0,0.3,0.3,200]))#,sigma=np.sqrt(vals),absolute_sigma=True)

        mpv_data = popt[0]
        mpv_err  = np.sqrt(np.diag(popv))[0]
        
        #print 'norm : %f and N entries : %i'%(popt[-1],dftmp.shape[0])
        
        mpv_theory = DPDX(ERange(rravg),0.35,105.6)
        
        #print '\t @ gain %i w/ RR %.0f ->  MPV measured : %.02f \t theory : %.02f \t err : %.02f'\
        #%(elecgain,rravg,mpv_data,mpv_theory,mpv_err)
        
        chilocal += ( (mpv_data-mpv_theory)**2 ) / (mpv_err**2 + 0.015**2 + 0.01**2)
        
    print 'Gain = %.02f -> chisq = %.02f'%(elecgain,chilocal)
    
    chisq_v.append(chilocal/float(len(rr_ranges)-1.))

In [ ]:
chi_min = 100000.
gain_min = 0.
for i,chi in enumerate(chisq_v):
    
    if (chi < chi_min):
        
        chi_min  = chi
        gain_min = elec_gain[i]


fig = plt.figure(figsize=(10,6))

min1sigma = 1000
max1sigma = 0
for i,g in enumerate(elec_gain):
    chi = chisq_v[i]
    chidiff = chi - chi_min
    if (chidiff < 1):
        if (g > max1sigma): max1sigma = g
        if (g < min1sigma): min1sigma = g

plt.plot(elec_gain,chisq_v,'bo--')
plt.xlabel('Electronics Gain [$e^-$/ADC]',fontsize=20,fontweight='bold')
plt.ylabel('$\chi^2$ / d.o.f.',fontsize=20,fontweight='bold')
plt.axvline(gain_min,color='k',lw=3,label='$\chi^2$ = %.02f @ gain = %.01f $e^-$/ADC'%(chi_min,gain_min))
plt.title(PLOTSTATUS, loc="right", fontsize=14)
plt.grid()
# plt.title("MicroBooNE Preliminary Run4a Data", loc="right", color="darkgrey")
# plt.xlim([245,257])
# plt.ylim([0,30])
# plt.ylim([0,15])
plt.axhspan(chi_min,chi_min+1,color='k',alpha=0.2,label='+1 $\sigma$ interval : [%.01f,%.01f]'%(min1sigma,max1sigma))
plt.legend(loc=1,fontsize=18)
plt.savefig(FIGPATH + "chi2_v_egain.png",bbox_inches='tight')
plt.savefig(FIGPATH + "chi2_v_egain.pdf",bbox_inches='tight')

# plt.savefig(FIGPATH + "chi2_v_egain_clipped.png",bbox_inches='tight')
# plt.savefig(FIGPATH + "chi2_v_egain_clipped.pdf",bbox_inches='tight')
plt.show()
print gain_min

In [ ]:
data["dedx"] = data.apply(lambda x: dEdx(x, gain_min), axis=1)

In [ ]:
print(data["rr"].min(), data["rr"].max())

In [ ]:
fig = plt.figure(figsize=(10, 6))
BINS = (np.linspace(0, 150, 100), np.linspace(1, 4, 100))
plt.hist2d(data["rr"].values, data["dedx"].values, bins=BINS)
plt.grid()
plt.xlabel("Residual Range [cm]", fontsize=18, fontweight="bold")
plt.ylabel("dE/dx [MeV / cm]", fontsize=18, fontweight="bold")
plt.title(
    "Tagged Stopping Muon Profile", fontsize=18, fontweight="bold", loc="left"
)
plt.title(PLOTSTATUS, fontsize=14, loc="right")
plt.savefig(FIGPATH + "rr_v_dqdx.png", bbox_inches="tight")
plt.show()